In [33]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("airline dataset analysis").getOrCreate()

In [34]:
from pyspark.sql.types import Row
from datetime import datetime

In [35]:
pwd

'/home/ankita/sparkCourse/spark2'

In [36]:
flight_path = 'flights.csv'
airline_path = 'airlines.csv'
airport_path = 'airports.csv'

In [37]:
flights = spark.read.format('csv').option('header', 'true').load(flight_path)
airline = spark.read.format('csv').option('header', 'true').load(airline_path)
airport = spark.read.format('csv').option('header', 'true').load(airport_path)

In [38]:
flights.show(5)

+----------+--------+-------------+------+-----------+---------+---------------+-------+-------------+--------+--------+
|      date|airlines|flight_number|origin|destination|departure|departure_delay|arrival|arrival_delay|air_time|distance|
+----------+--------+-------------+------+-----------+---------+---------------+-------+-------------+--------+--------+
|2014-04-01|   19805|            1|   JFK|        LAX|     0854|          -6.00|   1217|         2.00|  355.00| 2475.00|
|2014-04-01|   19805|            2|   LAX|        JFK|     0944|          14.00|   1736|       -29.00|  269.00| 2475.00|
|2014-04-01|   19805|            3|   JFK|        LAX|     1224|          -6.00|   1614|        39.00|  371.00| 2475.00|
|2014-04-01|   19805|            4|   LAX|        JFK|     1240|          25.00|   2028|       -27.00|  264.00| 2475.00|
|2014-04-01|   19805|            5|   DFW|        HNL|     1300|          -5.00|   1650|        15.00|  510.00| 3784.00|
+----------+--------+-----------

In [39]:
airline.createOrReplaceTempView('airlines')

In [40]:
sqlc = SQLContext(sc)
sqlc

In [45]:
sql_airline = sqlc.sql('select * from airlines')
sql_airline.columns

['Code', 'Description']

In [47]:
flights.createOrReplaceTempView('flight')
sql_flight = sqlc.sql('select * from flight')
sql_flight.columns

['date',
 'airlines',
 'flight_number',
 'origin',
 'destination',
 'departure',
 'departure_delay',
 'arrival',
 'arrival_delay',
 'air_time',
 'distance']

In [48]:
[sql_flight.count(), sql_airline.count()]

[476881, 1579]

In [49]:
flights_cnt =  sqlc.sql('select count(*) from flight')
airline_cnt =  sqlc.sql('select count(*) from airlines')\


In [50]:
flights_cnt, airline_cnt

(DataFrame[count(1): bigint], DataFrame[count(1): bigint])

In [51]:
flights_cnt.show()

+--------+
|count(1)|
+--------+
|  476881|
+--------+



In [52]:
airline_cnt.show()

+--------+
|count(1)|
+--------+
|    1579|
+--------+



In [54]:
distance = sqlc.sql('select * from flight').agg({'distance':'sum'}).withColumnRenamed('sum(distance)', 'sum_distance')

In [55]:
distance.columns


['sum_distance']

In [59]:
distance.show()

+------------+
|sum_distance|
+------------+
|3.79052917E8|
+------------+



In [66]:
#delays for year 2012
all_delays_2012 = sqlc.sql('select date,airlines from flight where departure_delay > 0 and year(date) = 2012')

In [67]:
all_delays_2012.show()

+----+--------+
|date|airlines|
+----+--------+
+----+--------+



In [69]:
#delays for year 2014
all_delays_2014 = sqlc.sql('select date,airlines,departure_delay from flight where departure_delay > 0 and year(date) = 2014')
all_delays_2014.show(5)

+----------+--------+---------------+
|      date|airlines|departure_delay|
+----------+--------+---------------+
|2014-04-01|   19805|          14.00|
|2014-04-01|   19805|          25.00|
|2014-04-01|   19805|         126.00|
|2014-04-01|   19805|         125.00|
|2014-04-01|   19805|           4.00|
+----------+--------+---------------+
only showing top 5 rows



In [70]:
all_delays_2014.createOrReplaceTempView("all_delays")
all_delays_2014.orderBy(all_delays_2014.departure_delay.desc()).show(5)

+----------+--------+---------------+
|      date|airlines|departure_delay|
+----------+--------+---------------+
|2014-04-18|   20366|          99.00|
|2014-04-18|   20366|          99.00|
|2014-04-17|   19393|          99.00|
|2014-04-18|   19977|          99.00|
|2014-04-18|   20409|          99.00|
+----------+--------+---------------+
only showing top 5 rows



In [71]:
delay_cnt = sqlc.sql('select count(departure_delay) from all_delays')
delay_cnt.show(5)

+----------------------+
|count(departure_delay)|
+----------------------+
|                179015|
+----------------------+



In [72]:
percentageDelay = delay_cnt.collect()[0][0]/flights_cnt.collect()[0][0] * 100
percentageDelay

37.53871510922012

In [75]:
avg_dep_delay = sqlc.sql('select airlines,departure_delay from flight ')\
 .groupBy('airlines').agg({'departure_delay' : 'avg'}).withColumnRenamed('avg(departure_delay)', 'dep_delay')

In [77]:
avg_dep_delay.orderBy(avg_dep_delay.dep_delay.desc()).show(5) #top 5 with most departure delay

+--------+------------------+
|airlines|         dep_delay|
+--------+------------------+
|   19393|13.429567657134724|
|   20366|12.296210112379818|
|   19977| 8.818392620527979|
|   20436| 8.716275167785234|
|   20409|  8.31110357194785|
+--------+------------------+
only showing top 5 rows

